# Setup

In [2]:
import pandas as pd
from pathlib import Path
import numpy as np
from datetime import datetime, timedelta
from tqdm import tqdm
tqdm.pandas()

In [1]:
# https://usafacts.org/data/topics/people-society/population-and-demographics/population-data/population/

# Traffic data

In [78]:
traffic_table = (
    pd.concat(
        [
            pd.read_csv(Path("./raw_data/traffic/world_traffic.csv")),
            pd.read_csv(Path("./raw_data/traffic/usa_traffic.csv")),
        ]
    )[lambda df: df["Metric"] == "Organic Traffic"]
    .drop(columns=["Target", "Target Type", "Summary", "Metric"])
    .set_index("Database")
    .T.rename(
        columns={"Worldwide": "traffic_world", "us": "traffic_usa", "index": "date"}
    )
    .assign(date=lambda df: pd.to_datetime(df.index))
)

In [79]:
imputated_date = np.arange(
    pd.to_datetime("2007-4"),
    traffic_table.index.min(),
    np.timedelta64(1, "M"),
    dtype="datetime64[M]",
).astype(datetime)

imputated_world = np.arange(
    traffic_table['traffic_world'][0] / (len(imputated_date) + 1),
    traffic_table['traffic_world'][0],
    traffic_table['traffic_world'][0] / (len(imputated_date) + 1)
)

imputated_usa = np.arange(
    traffic_table['traffic_usa'][0] / (len(imputated_date) + 1),
    traffic_table['traffic_usa'][0],
    traffic_table['traffic_usa'][0] / (len(imputated_date) + 1)
)

In [80]:
traffic_table = (
    traffic_table.append(
        pd.DataFrame(
            {
                "date": imputated_date,
                "traffic_world": imputated_world,
                "traffic_usa": imputated_usa,
            }
        )
    )
    .assign(date=lambda df: pd.to_datetime(df["date"]))
    .set_index("date")
    .sort_index()
    .pipe(
        lambda table: table.join(
            table.index.to_series()
            .apply(lambda t: table[lambda df: df.index >= t].sum())
            .rename(columns={"traffic_world": "cum_world", "traffic_usa": "cum_usa"})
        )
    )
    .astype(np.int64)
    .assign(
        year=lambda df: df.index.to_series().dt.year,
        month=lambda df: df.index.to_series().dt.month,
    )
)

In [81]:
traffic_table.to_parquet(Path("./data/traffic_data.parquet"))

# GSS data

In [6]:
USE_COLS = [
    "id",
    "year",
    "wkharsex",
    "relig",
    "age",
    "race",
    "sexeduc",
    "xmovie",
    "polviews",
    "partners",
    "condom",
    "relatsex",
    "wrkstat",
    "marital",
    "sex",
    "homosex",
    "harsexjb",
    "paidsex",
    'wwwhr',
    'pornlaw',
    'relgendr',
    'sexfreq',
    'sexornt'
]

In [15]:
gss_table = (
    pd.read_stata(Path('./raw_data/gss/gss7221_r1a.dta'), columns = USE_COLS)
    [lambda df: (2007 <= df['year']) & (df['year'] <= 2022)]
)

In [17]:
gss_table.to_stata(Path('./data/gss_data.dta'))

/home/psebi_eury/.local/lib/python3.8/site-packages/pandas/io/stata.py:660: ValueLabelTypeMismatch: 
Stata value labels (pandas categories) must be strings. Column age contains
non-string labels which will be converted to strings.  Please check that the
Stata data file created has not lost information due to duplicate labels.

  warnings.warn(
/home/psebi_eury/.local/lib/python3.8/site-packages/pandas/io/stata.py:660: ValueLabelTypeMismatch: 
Stata value labels (pandas categories) must be strings. Column wwwhr contains
non-string labels which will be converted to strings.  Please check that the
Stata data file created has not lost information due to duplicate labels.

  warnings.warn(


# Video data

In [61]:
video_table = pd.read_parquet(Path('./raw_data/video/video_data.parquet'))

In [62]:
video_table = (
    video_table
    [['view', 'date']]
    .assign(date_m = lambda df: df['date'].progress_apply(lambda x: pd.to_datetime(f'{x.year}-{x.month}' if not pd.isna(x) else float('nan'))))
)

  0%|          | 0/3401962 [00:10<?, ?it/s]


KeyboardInterrupt: 

In [ ]:
video_table = (
    video_table
    [lambda df: ~df["date"].isna()]
    .assign(
        cum_traffic = lambda df: df["date_m"].progress_apply(lambda x: traffic_table["cum_world"][x])
    )
    .assign(view_per_traffic = lambda df: df['view'] / df['cum_traffic'])
    .drop(columns = ['cum_traffic'])
    .assign(year = lambda df: df.date_m.dt.year)
)

In [71]:
video_table = pd.read_parquet(Path('./data/video_data.parquet'))

# Population data

In [95]:
(
    pd.read_csv(Path("./raw_data/population/POPTHM.csv"))
    .assign(
        DATE=lambda df: pd.to_datetime(df["DATE"]),
        year=lambda df: df["DATE"].dt.year,
        month=lambda df: df["DATE"].dt.month,
    )
    .rename(columns={"DATE": "date", "POPTHM": "population"})
    .set_index("date")
#     .to_parquet(Path("./data/population_data.parquet"))
)

,population,year,month
date,,,
1959-01-01,175818.0,1959,1
1959-02-01,176044.0,1959,2
1959-03-01,176274.0,1959,3
1959-04-01,176503.0,1959,4
1959-05-01,176723.0,1959,5
...,...,...,...
2021-06-01,331297.0,2021,6
2021-07-01,331395.0,2021,7
2021-08-01,331507.0,2021,8
